<a href="https://colab.research.google.com/github/Vishvam10/Agar.io-Clone/blob/master/RAG_Mistral_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Imports and Installs**

In [ ]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

import warnings
warnings.filterwarnings("ignore")

### **Model Initialization**

In [ ]:
#################################################################
# Tokenizer and bitsandbytes parameters
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

#### **RAG Setup**

In [ ]:
!playwright install
!playwright install-deps

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = [
  'https://neobrutalism-components.vercel.app/docs/components/Accordion',
  'https://neobrutalism-components.vercel.app/docs/components/Alert',
  'https://neobrutalism-components.vercel.app/docs/components/Avatar',
  'https://neobrutalism-components.vercel.app/docs/components/Badge',
  'https://neobrutalism-components.vercel.app/docs/components/Button',
  'https://neobrutalism-components.vercel.app/docs/components/Card',
  'https://neobrutalism-components.vercel.app/docs/components/Checkbox',
  'https://neobrutalism-components.vercel.app/docs/components/Drawer',
  'https://neobrutalism-components.vercel.app/docs/components/Dropdown',
  'https://neobrutalism-components.vercel.app/docs/components/ImageCard',
  'https://neobrutalism-components.vercel.app/docs/components/Input',
  'https://neobrutalism-components.vercel.app/docs/components/Marquee',
  'https://neobrutalism-components.vercel.app/docs/components/Modal',
  'https://neobrutalism-components.vercel.app/docs/components/Newsletter',
  'https://neobrutalism-components.vercel.app/docs/components/RadioGroup',
  'https://neobrutalism-components.vercel.app/docs/components/Select',
  'https://neobrutalism-components.vercel.app/docs/components/Tabs',
  'https://neobrutalism-components.vercel.app/docs/components/Textarea',
  'https://neobrutalism-components.vercel.app/docs/components/Tooltip'
]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [ ]:
# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=512,
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

In [ ]:
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e1d340f6b60>


### **Prompt Templating**

In [ ]:
prompt_template = """
### [INST] Instruction: You are a UI generating assistant. You'll carefully analyse the input prompt and then create the UI using React and CSS3. Use CSS classes wherever you can and define them in a styles tag.
Infer things like background colors, shadows, borders, etc from the nature of the UI component. Follow the input prompt thoroughly. Print only the code that is asked in the input prompt
and nothing else (no explanation or comments or things like "here you go", "here's the code that you asked for", etc)

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

### **Test Output**

##### **Without RAG**

In [ ]:
question = '''
Generate a simple card with a heading and some sample text (use lorem ipsum).
Give the react component as output and follow the styles given as context.
'''

context = ''

result = llm_chain.invoke({"context": context, "question": question})

print(result['text'].replace('\n', ''))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```jsximport React from'react';import './styles.css';const Card = ({ heading, text }) => {  return (    <div className="card">      <h2 className="heading">{heading}</h2>      <p className="text">{text}</p>    </div>  );};export default Card;```


##### **With RAG**

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple card with a heading and some sample text. Infer from the context the heading and a sample text.
Give the react component as output and follow the styles given as context.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### **UI Generation**

#### **Card Component**

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple image card. The top of the card should have an image with source as : https://a-z-animals.com/media/2022/11/shutterstock_606517310-1024x650.jpg.
The bottom of the card should contain the text "Sample Image". Give the react component as output and follow the styles given in the context.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```javascript
import React from'react';
import './styles.css';

const ImageCard = ({ imageUrl }) => {
  return (
    <figure className="w-[250px] overflow-hidden rounded-md border-2 border-black bg-[#bc95d4] font-bold shadow-[4px_4px_0px_0px_rgba(0,0,0,1)]">
      <img className="w-full" src={imageUrl} alt="image" />
      <figcaption className="border-t-2 border-black p-4">
        Sample Image
      </figcaption>
    </figure>
  );
};

export default ImageCard;
```


In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple image card. The top of the card should have an image with source as : https://a-z-animals.com/media/2022/11/shutterstock_606517310-1024x650.jpg.
The bottom of the card should contain the text "Sample Image". Give the react component as output and follow the styles given in the context. DO NOT USE ANY PROPS
instead hardcode the values wherever necessary.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```javascript
import React from'react';
import './styles.css';

const ImageCard = () => {
  return (
    <figure className="w-[250px] overflow-hidden rounded-md border-2 border-black bg-[#bc95d4] font-bold shadow-[4px_4px_0px_0px_rgba(0,0,0,1)]">
      <img className="w-full" src="https://a-z-animals.com/media/2022/11/shutterstock_606517310-1024x650.jpg" alt="image" />
      <figcaption className="border-t-2 border-black p-4">
        Sample Image
      </figcaption>
    </figure>
  );
};

export default ImageCard;
```


In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple card. The top of the card should have an image with source as : https://a-z-animals.com/media/2022/11/shutterstock_606517310-1024x650.jpg.
The bottom of the card should contain the text "Sample Image". Below that, there should be a smaller text (1.5 times smaller) that says "In publishing and graphic design, Lorem ipsum
is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content." Below that, there should be
a simple button that says 'Read More'.

Give the react component as output and follow the styles given in the context. DO NOT USE ANY PROPS
instead hardcode the values wherever necessary.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```javascript
import React from'react';
import './styles.css';

const Card = () => {
  return (
    <div className="w-[250px] rounded-md border-2 border-black bg-[#bc95d4] font-bold shadow-[4px_4px_0px_0px_rgba(0,0,0,1)]">
      <div className="border-b-2 border-black p-4">
        <img className="w-full" src="https://a-z-animals.com/media/2022/11/shutterstock_606517310-1024x650.jpg" alt="image" />
      </div>
      <div className="p-4">
        <h2 className="text-lg">Sample Image</h2>
        <p className="text-sm">In publishing and graphic design, Lorem ipsum is a placeholder text commonly used to demonstrate the visual form of a document or a typeface without relying on meaningful content.</p>
        <button className="bg-[#bc95d4] text-white font-bold py-2 px-4 rounded-md hover:bg-[#8c7aaa]">Read More</button>
      </div>
    </div>
  );
};

export default Card;
```


In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple product card. The top of the card should have an image with source as : https://shorturl.at/kmzN2. For the bottom part, use a 2-column layout.
The bottom of card should contain the following : "Sample Shoes" as heading, "Sample Company" as subheading in a lighter text and "$120.00" as price
in a bold font. All of them should be below each other and should be in the first column. In the second column, right beside the heading, include a button that says "+".

Give the react component as output and follow the styles given in the context. DO NOT USE ANY PROPS instead hardcode the values wherever necessary.
'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```javascript
import React from'react';

const ProductCard = () => {
  return (
    <div className="w-[250px] overflow-hidden rounded-md border-2 border-black bg-[#bc95d4] font-bold shadow-[4px_4px_0px_0px_rgba(0,0,0,1)]">
      <img className="w-full" src="https://shorturl.at/kmzN2" alt="image" />
      <div className="p-4">
        <h2 className="text-lg">Sample Shoes</h2>
        <div className="mt-2">
          <p className="text-sm">Sample Company</p>
          <p className="text-sm font-bold">$120.00</p>
        </div>
        <button className="bg-white border-2 border-black p-2 rounded-md mt-2">+</button>
      </div>
    </div>
  );
};

export default ProductCard;
```


#### **Input Component**

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple form component. Give the form a title (heading) of "Contact Form". This should follow a 12 column layout. Inside this form component have the following in separate rows :
- input component with placeholder as "Sample" and label as "Name". It should spans 12 columns
- input component with placeholder as "sample@gmail.com" and label as "Email". It should spans 12 columns
- one button component with text "Submit" at the bottom of the container.

Make the form responsive and make sure maximum dimensions are height = 800px and width = 500px.

Give the react component (JS not typescript) as output and follow the styles given in the context.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.
Define all the functions that you use in the component definition.
DO NOT USE ANY OTHER ADDITIONAL LIBRARIES for icons, etc.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```jsx
import React, { useState } from'react';

const ContactForm = () => {
  const [name, setName] = useState('');
  const [email, setEmail] = useState('');

  const handleSubmit = (e) => {
    e.preventDefault();
    console.log('Submitted!');
  };

  return (
    <div className="max-h-800 max-w-500 mx-auto">
      <div className="flex flex-col gap-4">
        <div className="flex flex-row gap-2">
          <label htmlFor="name" className="font-bold text-gray-700">
            Name
          </label>
          <input
            type="text"
            id="name"
            className="flex-grow rounded-md border-2 border-black p-2 font-bold shadow-[4px_4px_0px_0px_rgba(0,0,0,1)]"
            placeholder="Sample"
            value={name}
            onChange={(e) => setName(e.target.value)}
          />
        </div>
        <div className="flex flex-row gap-2">
          <label htmlFor="email" className="font-bold text-gray-700">
            Email
          </label>
          <inpu

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a simple form component. Give the form a title (heading) of "Contact Form". This should follow a 12 column layout. Inside this form component have the following :
first row :
- input component with placeholder as "Sample" and label as "Name". It should spans 6 columns
- input component with placeholder as "sample@gmail.com" and label as "Email". It should spans 6 columns
second row :
- input component with placeholder as "Sample Address" and label as "Address". It should spans 8 columns
- input component with placeholder as "Sample PIN" and label as "Pincode". It should spans 4 columns
last row :
- one button component with text "Submit". It should span 12 columns.

Make the form responsive and make sure maximum dimensions are height = 800px and width = 500px.

Give the react component (JS not typescript) as output and follow the styles given in the context.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.
DO NOT USE ANY OTHER ADDITIONAL LIBRARIES for icons, etc.
Define all the functions that you use in the component definition.

'''

result = rag_chain.invoke(question)

print(result['text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```javascript
import React, { useState } from'react';

const ContactForm = () => {
  const [name, setName] = useState('');
  const [email, setEmail] = useState('');
  const [address, setAddress] = useState('');
  const [pincode, setPincode] = useState('');

  const handleSubmit = (e) => {
    e.preventDefault();
    console.log({ name, email, address, pincode });
  };

  return (
    <div className="max-h-800 max-w-500 mx-auto">
      <div className="flex flex-col gap-4">
        <div className="flex flex-row gap-4">
          <div className="flex flex-col">
            <label htmlFor="name">Name</label>
            <input
              type="text"
              id="name"
              placeholder="Sample"
              className="w-full rounded-md p-2 border-2 border-black font-bold shadow-[4px_4px_0px_0px_rgba(0,0,0,1)]"
              value={name}
              onChange={(e) => setName(e.target.value)}
            />
          </div>
          <div className="flex flex-col">
       

Complex layouts for some reason messes up the output. Probably need better examples while fine-tuning it with RAG.


In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

question = '''
Generate a container component. This should follow a 12 column layout.  Inside this container component, there should be 4 rows.
first row contains the following :
- one inpit component with dummy placeholder and label as "Label". It should span 12 columns
second row contains the following :
- two inpit components with dummy placeholder and label as "Label". Each should span 6 columns
third row contains the following :
- three inpit components  with dummy placeholder and label as "Label". Each should span 4 columns

Create all the components according to the context and use them here.
Make the form responsive and make sure maximum dimensions are height = 800px and width = 500px.

Give the react component (JS not typescript) as output and follow the styles given in the context.
DO NOT USE ANY PROPS instead hardcode the values wherever necessary.
DO NOT USE ANY OTHER ADDITIONAL LIBRARIES for icons, components, etc.
DO NOT IMPORT ANY COMPONENTS.
Define all the functions that you use in the component definition.

'''

result = rag_chain.invoke(question)

print(result['text'])